# **Importing Dependencies**

In [ ]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# **Downloading dataset from kaggle**

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 19% 5.00M/25.7M [00:00<00:00, 37.0MB/s]
100% 25.7M/25.7M [00:00<00:00, 128MB/s] 


In [ ]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

# **Data Processing**

In [ ]:
data = pd.read_csv("IMDB Dataset.csv")

In [ ]:
data.shape

(50000, 2)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# **Checking the label distribution**

In [ ]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


**So it is a balanced dataset**

# **Converting catagorical to numerical**

In [ ]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

<ipython-input-10-c1a779bd745a>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


# **Train-Test Splitting**

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# **Checking the shape**

In [ ]:
print("Shape of train data:", train_data.shape)
print("Shape of test data:", test_data.shape)

Shape of train data: (40000, 2)
Shape of test data: (10000, 2)


# **Lets's Tokenize the data**

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [ ]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [ ]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

# **Let's build  LSTM model**

In [ ]:
model=Sequential()
model.add(Embedding(5000, 128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


# **Fitting the model**

In [ ]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 135s 249ms/step - accuracy: 0.7022 - loss: 0.5488 - val_accuracy: 0.8249 - val_loss: 0.3932
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 132s 245ms/step - accuracy: 0.8503 - loss: 0.3578 - val_accuracy: 0.8626 - val_loss: 0.3402
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 144s 249ms/step - accuracy: 0.8760 - loss: 0.3053 - val_accuracy: 0.7960 - val_loss: 0.4350
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 124s 248ms/step - accuracy: 0.8821 - loss: 0.2935 - val_accuracy: 0.8636 - val_loss: 0.3436
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 128s 255ms/step - accuracy: 0.9047 - loss: 0.2394 - val_accuracy: 0.8635 - val_loss: 0.3328


# **Evaluating the model**

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 84ms/step - accuracy: 0.8686 - loss: 0.3163
Test Loss: 0.3158298432826996
Test Accuracy: 0.8708000183105469


# **Let's check the model**

In [ ]:
def predicting(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [ ]:
review = "This movie was not so interesting."
sentiment = predicting(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step
The sentiment of the review is: negative


In [ ]:
review = "This movie was very amazing."
sentiment = predicting(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
The sentiment of the review is: positive


In [ ]:
review = "This is a boring movie ."
sentiment = predicting(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
The sentiment of the review is: negative


In [ ]:
review = "This is a good movie ."
sentiment = predicting(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
The sentiment of the review is: positive
